# **Stepwise Breakdown of the Near-Linear SSSP Algorithm**
This algorithm computes **Single-Source Shortest Paths (SSSP) with negative weights** efficiently using **graph decomposition, recursive scaling, and reweighted Dijkstra’s algorithm**.

## **1. Graph Representation**
- The graph is stored using an **adjacency list** representation.
- Each edge `(u, v, w)` represents a **directed weighted edge** from `u` to `v` with weight `w`.

---

## **2. Decompose the Graph Using Strongly Connected Components (SCCs)**
- Compute **Strongly Connected Components (SCCs)** using **Kosaraju’s or Tarjan’s algorithm**.
- Each SCC is treated as a **single unit** in the graph.
- **Purpose:** Helps in handling **negative cycles** effectively.

---

## **3. Compute a Price Function Using Recursive Scaling**
- The algorithm defines a **price function φ(v)** for every node `v` to adjust edge weights.
- The price function is computed **recursively** by:
  1. **Dividing edge weights by 2 at each recursion level** (logarithmic scaling).
  2. **Processing smaller subgraphs iteratively.**
- **Goal:** Ensure that **modified edge weights remain non-negative**.

---

## **4. Reweight the Graph Using the Price Function**
- Each edge weight is adjusted
- This ensures that **all edge weights become non-negative**, allowing **Dijkstra’s algorithm to be used** efficiently.

---

## **5. Run Dijkstra’s Algorithm on the Transformed Graph**
- Since the modified graph has **only non-negative weights**, we run **Dijkstra’s algorithm** (O((V + E) log V)) to compute shortest paths.

---

## **6. Restore the Original Shortest Path Values**
- The original shortest path distances are recovered by adjusting the final distances using the price function.
---

## **Time Complexity Analysis**
- **SCC Decomposition:** O(V + E)
- **Recursive Scaling:** O(log W) recursive calls
- **Graph Reweighting:** O(V + E)
- **Dijkstra on Adjusted Graph:** O((V + E) log V)
- **Total Complexity:** 

  \[
  O(m \log^3 n)
  \]


In [5]:
import heapq
import random
from collections import defaultdict, deque

class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = defaultdict(list)
    
    def add_edge(self, u, v, w):
        self.graph[u].append((v, w))
    
    def bellman_ford(self, src):
        """ Modified Bellman-Ford algorithm with path tracking."""
        dist = {i: float('inf') for i in range(self.V)}
        dist[src] = 0
        
        for _ in range(self.V - 1):
            for u in self.graph:
                for v, w in self.graph[u]:
                    if dist[u] + w < dist[v]:
                        dist[v] = dist[u] + w
        
        return dist
    
    def dijkstra(self, src):
        """Dijkstra's algorithm for non-negative edge weights."""
        pq = [(0, src)]
        dist = {i: float('inf') for i in range(self.V)}
        dist[src] = 0
        
        while pq:
            d, u = heapq.heappop(pq)
            if d > dist[u]:
                continue
            for v, w in self.graph[u]:
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    heapq.heappush(pq, (dist[v], v))
        
        return dist
    
    def strongly_connected_components(self):
        """Finds strongly connected components using Kosaraju’s algorithm."""
        def dfs(v, visited, stack):
            visited.add(v)
            for neighbor, _ in self.graph[v]:
                if neighbor not in visited:
                    dfs(neighbor, visited, stack)
            stack.append(v)
        
        def reverse_graph():
            rev_graph = Graph(self.V)
            for u in self.graph:
                for v, w in self.graph[u]:
                    rev_graph.add_edge(v, u, w)
            return rev_graph
        
        stack = []
        visited = set()
        for v in range(self.V):
            if v not in visited:
                dfs(v, visited, stack)
        
        reversed_graph = reverse_graph()
        visited.clear()
        sccs = []
        
        while stack:
            v = stack.pop()
            if v not in visited:
                scc = []
                dfs(v, visited, scc)
                sccs.append(scc)
        
        return sccs
    
    def scale_down(self, B):
        """Recursive scaling algorithm for handling negative weights."""
        if B <= 2:
            return {v: 0 for v in range(self.V)}
        
        SCCs = self.strongly_connected_components()
        price_function = {v: 0 for v in range(self.V)}
        
        for scc in SCCs:
            subgraph = Graph(len(scc))
            node_map = {node: i for i, node in enumerate(scc)}
            
            for u in scc:
                for v, w in self.graph[u]:
                    if v in node_map:
                        subgraph.add_edge(node_map[u], node_map[v], w)
            
            phi = subgraph.scale_down(B // 2)
            for node in scc:
                price_function[node] = phi[node_map[node]]
        
        return price_function

    def shortest_paths(self, src):
        """Main function to compute shortest paths with negative weights."""
        B = 2 * self.V  # Scaling factor
        price_function = self.scale_down(B)
        adjusted_graph = Graph(self.V)
        
        for u in self.graph:
            for v, w in self.graph[u]:
                adjusted_w = w + price_function[u] - price_function[v]
                adjusted_graph.add_edge(u, v, max(0, adjusted_w))
        
        return adjusted_graph.dijkstra(src)

# Create a sample graph
g = Graph(6)
g.add_edge(0, 1, -2)
g.add_edge(1, 2, 7)
g.add_edge(2, 3, 2)
g.add_edge(3, 1, -6)  # Cycle
g.add_edge(3, 4, 4)
g.add_edge(4, 5, 1)

# Run the algorithm
shortest_paths = g.shortest_paths(0)
print("Shortest Paths:", shortest_paths)


Shortest Paths: {0: 0, 1: 0, 2: 7, 3: 9, 4: 13, 5: 14}


In [6]:
# Updating DebugGraph to inherit all methods from Graph correctly

class DebugGraph(Graph):
    """Debugging graph class that extends Graph to add debug outputs."""
    
    def shortest_paths_with_debug(self, src):
        """Main function to compute shortest paths with debug outputs."""
        B = 2 * self.V  # Scaling factor
        price_function = self.scale_down(B)
        
        print("\nPrice Function: ", price_function)
        
        adjusted_graph = DebugGraph(self.V)

        # Store adjusted weights for debugging
        adjusted_weights = {}

        for u in self.graph:
            for v, w in self.graph[u]:
                adjusted_w = w + price_function[u] - price_function[v]
                adjusted_graph.add_edge(u, v, max(0, adjusted_w))
                adjusted_weights[(u, v)] = max(0, adjusted_w)

        print("\nAdjusted Graph Weights: ", adjusted_weights)

        # Run Dijkstra on the adjusted graph
        return adjusted_graph.dijkstra(src)


# Create the graph and run both Bellman-Ford and the ScaleDown-based approach
debug_graph = DebugGraph(6)
debug_graph.add_edge(0, 1, -2)
debug_graph.add_edge(1, 2, 7)
debug_graph.add_edge(2, 3, 2)
debug_graph.add_edge(3, 1, -6)  # Cycle
debug_graph.add_edge(3, 4, 4)
debug_graph.add_edge(4, 5, 1)

# Run Bellman-Ford from source node 0
bellman_ford_result = debug_graph.bellman_ford(0)
print("\nBellman-Ford Shortest Paths: ", bellman_ford_result)

# Run ScaleDown-based shortest paths with debugging
shortest_paths_debug = debug_graph.shortest_paths_with_debug(0)
print("\nScaleDown Algorithm Shortest Paths: ", shortest_paths_debug)



Bellman-Ford Shortest Paths:  {0: 0, 1: -2, 2: 5, 3: 7, 4: 11, 5: 12}

Price Function:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

Adjusted Graph Weights:  {(0, 1): 0, (1, 2): 7, (2, 3): 2, (3, 1): 0, (3, 4): 4, (4, 5): 1}

ScaleDown Algorithm Shortest Paths:  {0: 0, 1: 0, 2: 7, 3: 9, 4: 13, 5: 14}
